# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
file = "../output_data/weather_data.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Uberaba,-19.75,-47.93,27.26,28,0,4.18,BR,1601600103
1,Ushuaia,-54.80,-68.30,-1.00,92,75,9.30,AR,1601600088
2,Qaanaaq,77.48,-69.36,-9.85,79,82,2.24,GL,1601600143
3,Port Alfred,-33.59,26.89,14.30,71,97,9.08,ZA,1601600138
4,Albany,42.60,-73.97,12.78,70,23,1.81,US,1601600137


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [66]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# set humidity as weight
rating = weather_df["Humidity"].astype(float)

In [67]:
# Plot Map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
# reorganise dataframe and remove date
new_weather_df = pd.DataFrame(weather_df, columns = ["City","Country", "Max temp", "Wind Speed", "Cloudiness","Lat","Lng"])

# creating filters based on ideal weather criteria
max_temp = (new_weather_df["Max temp"] <= 32) & (new_weather_df["Max temp"] > 23)
wind_speed = new_weather_df["Wind Speed"] < 10
cloudiness = new_weather_df["Cloudiness"] == 0

# creating the shortlist based on the three filters
ideal_df = new_weather_df[max_temp & wind_speed & cloudiness]

In [69]:
ideal_df.head(10)

,City,Country,Max temp,Wind Speed,Cloudiness,Lat,Lng
0,Uberaba,BR,27.26,4.18,0,-19.75,-47.93
64,São Filipe,CV,27.33,4.03,0,14.90,-24.50
81,Camocim,BR,25.99,6.75,0,-2.90,-40.84
103,Chimoré,BO,27.43,1.42,0,-16.98,-65.13
104,La Santisima Trinidad,BO,29.55,1.50,0,-14.83,-64.90
110,Kiryat Gat,IL,23.89,0.89,0,31.61,34.76
113,Puerto Quijarro,BO,29.79,0.48,0,-17.78,-57.77
165,Aswān,EG,28.95,3.14,0,24.09,32.91
166,Chak Azam Sahu,PK,23.27,1.51,0,30.75,73.03
178,Bilma,NE,29.92,1.87,0,18.69,12.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
# setting up a new dataframe to work with
hotel_df = ideal_df

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Creating a loop to find hotels
for index, row in hotel_df.iterrows():
    city_name = row["City"]
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print(f"No hotel found for {city_name}... skipping.")

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No hotel found for Bilma... skipping.
No hotel found for Agadez... skipping.
No hotel found for Douentza... skipping.


In [73]:
# Remove cities with no values
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Country,Max temp,Wind Speed,Cloudiness,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
0,Uberaba,BR,27.26,4.18,0,-19.75,-47.93,Tamareiras Park Hotel,"R. Olegário Maciel, 187 - Centro, Uberaba",4.5
64,São Filipe,CV,27.33,4.03,0,14.90,-24.50,Hotel Ocean View & Restaurante Seafood,Bila Baixo Ilha do Fogo، São Filipe,4.5
81,Camocim,BR,25.99,6.75,0,-2.90,-40.84,Essenza Dune,"R. da Dunas, Jijoca de Jericoacoara",4.7
103,Chimoré,BO,27.43,1.42,0,-16.98,-65.13,Hotel de Selva El Puente,"Villa Tunari, Cochabamba",4.1
104,La Santisima Trinidad,BO,29.55,1.50,0,-14.83,-64.90,Hotel Río Ibare,Trinidad,4.0


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
names = hotel_df["City"]

In [60]:
# adding the info boxes to the map
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)

In [61]:
# adding the markers and displaying final map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))